# Results Number Analysis

In [1]:
import pandas as pd

## Experiment Config

In [2]:
year = "2019"
dl_year = "dl" + year

In [3]:
dl_ndcgeval = f'final-results/all.pass.{dl_year}.ndcgeval'
dl_treceval = f'final-results/all.pass.{dl_year}.treceval'

syndl_ndcgeval = f'final-results/all.pass.{dl_year}.SynDL.full.ndcgeval'
syndl_treceval = f'final-results/all.pass.{dl_year}.SynDL.full.treceval'

In [4]:
dl_result_ndcgeval = pd.read_csv(dl_ndcgeval, sep='\t', header=None, names=['run_id', 'metric', 'qid', 'score'])
dl_result_treceval = pd.read_csv(dl_treceval, sep='\t', header=None, names=['run_id', 'metric', 'qid', 'score'])

syndl_result_ndcgeval = pd.read_csv(syndl_ndcgeval, sep='\t', header=None, names=['run_id', 'metric', 'qid', 'score'])
syndl_result_treceval = pd.read_csv(syndl_treceval, sep='\t', header=None, names=['run_id', 'metric', 'qid', 'score'])

In [5]:
def get_cleaned_df(dataframe):
    # Filter out rows where 'qid' is 'all' and create a copy
    dataframe = dataframe[dataframe['qid'] != 'all'].copy()
    # Convert 'qid' to integer
    dataframe['qid'] = dataframe['qid'].astype(int)
    # Filter the dataframe based on specific metrics and create a copy
    dataframe = dataframe[dataframe['metric'].isin(['ndcg_cut_100', 'ndcg_cut_10', 'ndcg_cut_5', 'map'])].copy()
    # Convert 'score' to float64
    dataframe['score'] = dataframe['score'].astype('float64')
    # Group by 'run_id' and 'metric', and calculate the mean and count of 'score'
    dataframe = dataframe.groupby(['run_id', 'metric'])['score'].agg(['mean', 'count']).reset_index()
    # Drop the 'count' column
    dataframe = dataframe.drop('count', axis=1)
    # Pivot the table to have 'run_id' as index and 'metric' as columns
    dataframe = dataframe.pivot_table(index='run_id', columns='metric', values='mean').reset_index()
    return dataframe

In [6]:
dl_result_ndcgeval = get_cleaned_df(dl_result_ndcgeval)
dl_result_treceval = get_cleaned_df(dl_result_treceval)

In [7]:
dl_result = pd.merge(dl_result_ndcgeval, dl_result_treceval, on='run_id', how='inner')

In [8]:
syndl_result_ndcgeval = get_cleaned_df(syndl_result_ndcgeval)
syndl_result_treceval = get_cleaned_df(syndl_result_treceval)

In [9]:
syndl_result = pd.merge(syndl_result_ndcgeval, syndl_result_treceval, on='run_id', how='inner')

In [10]:
results = pd.merge(dl_result, syndl_result, on='run_id')

In [11]:
results = results.sort_values(by=['ndcg_cut_10_x'], ascending=False)

In [12]:
for eachline in results[0:5].itertuples(index=False):
    print(f"{eachline.run_id} & {round(eachline.ndcg_cut_5_x, 4)} & {round(eachline.ndcg_cut_10_x, 4)} & {round(eachline.ndcg_cut_100_x, 4)} & {round(eachline.map_x, 4)} && {round(eachline.ndcg_cut_5_y, 4)} & {round(eachline.ndcg_cut_10_y, 4)} & {round(eachline.ndcg_cut_100_y, 4)} & {round(eachline.map_y, 4)} \\\\")

idst_bert_p2 & 0.775 & 0.7632 & 0.6828 & 0.5193 && 0.6849 & 0.6771 & 0.6117 & 0.4713 \\
idst_bert_p3 & 0.7803 & 0.7594 & 0.6862 & 0.5307 && 0.679 & 0.6771 & 0.6152 & 0.4816 \\
p_exp_rm3_bert & 0.7427 & 0.7422 & 0.6745 & 0.5294 && 0.6674 & 0.6611 & 0.6065 & 0.4759 \\
p_bert & 0.7334 & 0.738 & 0.6585 & 0.5049 && 0.6637 & 0.656 & 0.6036 & 0.4774 \\
idst_bert_pr2 & 0.7637 & 0.7379 & 0.6357 & 0.4776 && 0.67 & 0.6595 & 0.5916 & 0.4716 \\
